In [21]:
import camelot

pdf_path = "content.pdf"

tables = camelot.read_pdf(pdf_path, pages="all", flavor='lattice')
print(f"Tổng cộng: {tables.n} bảng được tìm thấy.")

table_strings = []

for i, table in enumerate(tables):
    table_str = table.df.to_markdown(index=False)  # Dễ đọc hơn với index=False
    table_strings.append(table_str)
    print(f"\n--- Bảng {i + 1} ---")
    print(table_str)



Tổng cộng: 10 bảng được tìm thấy.

--- Bảng 1 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | Elected to the Hall of Fame on this ballot (named |
|     | in bold italics).                                 |
|     | Elected subsequently, as of 2025 (named in plain  |
|     | italics).                                         |
|     | Renominated for the 2019 BBWAA election by        |
|     | adequate performance on this ballot and has not   |
|     | subsequently been eliminated.                     |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expiration on subsequent      |

--- Bảng 2 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | ballots.                                          |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expira

In [22]:
import requests
import json

def generate_response(prompt):
    """
    Gửi prompt đến Ollama và nhận về một đoạn phản hồi hoàn chỉnh.
    
    Tham số:
        prompt (str): Câu hỏi hoặc yêu cầu bạn muốn gửi đến mô hình.

    Trả về:
        str: Phản hồi hoàn chỉnh từ mô hình.
    """
    full_response = ""
    context = f"""
   You are given a list of table fragments extracted from a PDF document using OCR. Some tables were split across multiple pages and therefore appear as separate fragments in the input.

    ### 📌 TIPS FOR TABLE MERGING:

    ✅ Merge when:
    - Same number of columns
    - Similar data types per column (e.g., names, numbers, percentages)
    - Content flows naturally or continues meaning

    ❌ Do NOT merge when:
    - Column counts differ
    - Data types mismatch (e.g., name + long description vs. name + vote count)
    - No logical connection between contents

    ---

    ### ✅ EXAMPLE 1 – Tables with matching data patterns

    #### Input

    --- Bảng A ---
    | Player       | Votes | Percent |
    |--------------|-------|---------|
    | Chipper Jones| 410   | 97.2%   |

    --- Bảng B ---
    | Player          | Votes | Percent |
    |------------------|-------|---------|
    | Vladimir Guerrero| 392   | 92.9%   |

    --- Bảng C ---
    | Candidate     | Category | Ref  |
    |---------------|----------|------|
    | Jack Morris   | Player   | [12] |

    #### Expected Output

    ### Merged Table: A + B
    | Player            | Votes | Percent |
    |-------------------|-------|---------|
    | Chipper Jones     | 410   | 97.2%   |
    | Vladimir Guerrero | 392   | 92.9%   |

    ### Original Table C
    | Candidate     | Category | Ref  |
    |---------------|----------|------|
    | Jack Morris   | Player   | [12] |

    ---

    ### ✅ EXAMPLE 2 – Tables without clear headers

    #### Input

    --- Bảng D ---
    | Name           | Year | Score |
    |----------------|------|-------|
    | Alice          | 2020 | 92    |
    | Bob            | 2020 | 88    |

    --- Bảng E ---
    | Carol          | 2020 | 95    |
    | David          | 2020 | 89    |

    #### Expected Output

    ### Merged Table: D + E
    | Name           | Year | Score |
    |----------------|------|-------|
    | Alice          | 2020 | 92    |
    | Bob            | 2020 | 88    |
    | Carol          | 2020 | 95    |
    | David          | 2020 | 89    |

    ---

    Now, here is your actual input list of table fragments:
    {prompt}
    """
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gemma3:latest",
        "prompt": context,
        "stream": True
    }

    with requests.post(url, json=payload, stream=True) as response:
        for line in response.iter_lines():
            if line:
                try:
                    data = json.loads(line)
                    if 'response' in data:
                        chunk = data['response']
                        full_response += chunk
                    if data.get('done', False):
                        break
                except json.JSONDecodeError:
                    continue  

    return full_response.strip()

In [23]:
all_tables_combined = ""

for i, table in enumerate(tables):
    table_str = table.df.to_markdown(index=False)  
    all_tables_combined += f"\n\n--- Bảng {i + 1} ---\n"
    all_tables_combined += table_str





In [24]:
print("\n--- Toàn bộ bảng đã gộp ---")
print(all_tables_combined)


--- Toàn bộ bảng đã gộp ---


--- Bảng 1 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | Elected to the Hall of Fame on this ballot (named |
|     | in bold italics).                                 |
|     | Elected subsequently, as of 2025 (named in plain  |
|     | italics).                                         |
|     | Renominated for the 2019 BBWAA election by        |
|     | adequate performance on this ballot and has not   |
|     | subsequently been eliminated.                     |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expiration on subsequent      |

--- Bảng 2 ---
| 0   | 1                                                 |
|:----|:--------------------------------------------------|
|     | ballots.                                          |
|     | Eliminated from annual BBWAA consideration by     |
|     | poor performance or expiration

In [25]:
if all_tables_combined.strip():
    prompt = all_tables_combined
    print(generate_response(prompt))

Okay, let's analyze the table fragments and determine the expected output based on the provided guidelines.

**Here's the breakdown and the expected merged table:**

**Analysis & Merging Strategy**

*   **Bảng 1:**  This table appears to be a single player's voting data.
*   **Bảng 2:** This table is a continuation of Bảng 1, adding more voting data.
*   **Bảng 3:** This table shows player names, and then the name Chipper Jones.
*   **Bảng 4:** This table includes player names, votes, percentages, and changes.  It represents a collection of players' voting data.
*   **Bảng 5:** This table has Vladimir Guerrero listed.
*   **Bảng 6:** This table has Vladimir Guerrero, and a variety of players listed with votes and percentages.
*   **Bảng 7:** This table lists Jim Thome
*   **Bảng 8:** This table lists Jack Morris.
*   **Bảng 9:** This table shows category data for various players.
*   **Bảng 10:** This table is a hall of fame list

**Expected Output**

The output should be a merged tabl